In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import math
import numpy as np

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [3]:
page = 1
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
page_url =  district.format(page)
page_url


'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

# Заходим на страничку поиска квартир

In [4]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [5]:
district


'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

# Будем получать ссылки на квартиры со страниц

page_url - ссылка на страницу в формате гиперссылки
requests.get - найти страницу. Ответ <Response [200]> обозначает, что страница найдна
search_page.content - заходит в код страницы
BeautifulSoup - приводит код страницы к более читаемому виду
findAll (attrs=) - как я поняла находит в коде места с такими уточнениями
re.split - разбивает строку по заданному Шаблону: т.е. у нас разбивает на информацию об отдельных квартирах
isdigit - проверяет состоит ли строка только из цифр

In [276]:

def getlinks(district,n_pages):
    links = []
    for page in range(1, n_pages+1): 
        page_url =  district.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
    return(links)
    

In [7]:
links=getlinks(district,n_pages)

In [8]:
n_flats= len(links)

#Будем теперь искать ссылки на каждую квартиру

In [57]:
def getFlatPage(link):
    flat_url = 'http://www.cian.ru/sale/flat/' + str(link) + '/'
    f_page = requests.get(flat_url)
    f_page = f_page.content
    f_page = BeautifulSoup(f_page, 'lxml')
    return(f_page)

In [285]:
flat_page=getFlatPage(links[0])


In [11]:
flatStats = {'District':0}

# Достать цену

In [274]:
def getPrice(flat_page):
    try:
        price = flat_page.find('div', attrs={'class':'object_descr_price'})
        price = re.split('<div>|руб|\W', str(price))
        price = "".join([i for i in price if i.isdigit()][-3:])
        price=int(price)
    except:
        price = np.nan
    return(price)

In [273]:
getPrice(flat_page)

12000000

In [14]:
flatStats['Price'] = getPrice(flat_page)

# Координаты

In [15]:
def getCoordinates(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return(lat,lon)

In [16]:
getCoordinates(flat_page)

(55.750013, 37.594081)

In [17]:
#flatStats['lat'], flatStats['lon'] = getCoordinates(flat_page)

# Функция, возвращающая расстояние от квартиры с данной странцы до центра

In [18]:
def CenterDistance(flat_page):
    lat, lon = getCoordinates(flat_page)
    EarthRad = 6371
    center_lat = 55.755826
    center_lon = 37.617300
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon, lat, center_lon, center_lat])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = EarthRad * c
    return(round(km,2) )   

In [19]:
CenterDistance(flat_page)

1.59

In [20]:
flatStats['Dist'] = CenterDistance(flat_page)

In [21]:
flatStats

{'Dist': 1.59, 'District': 0, 'Price': 88782540}

# Число комнат

In [286]:
def getRoomNumber(flat_page):
    try:
        rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
        rooms = html_stripper(rooms)
        room_number = ''
        for i in re.split('-|\n', rooms):
            if 'комн' in i:
                break
            else:
                room_number += i
        room_number = "".join(room_number.split())
        room_number=int(room_number)
    except:
        room_number=np.nan
    return(room_number)

In [287]:
getRoomNumber(flat_page)

nan

In [24]:
flatStats['Rooms'] = getRoomNumber(flat_page)


In [219]:
flatStats

{'Bal': 1,
 'Brick': 0,
 'Dist': 20.21,
 'District': 8,
 'Floor': 10,
 'Kitsp': 11.0,
 'Livesp': 46.0,
 'Metrdist': 13,
 'N': 1,
 'New': 0,
 'Nfloors': 17,
 'Price': 10500000,
 'Rooms': 3,
 'Tel': 0,
 'Totsp': 76.0,
 'Walk': 1,
 'lat': 55.852935,
 'lon': 37.343892}

# Ищем всю остальную информацию по квартире в табличке

In [26]:
def getTableOfVariables(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    return(table)

In [27]:
getTableOfVariables(flat_page)

'\n\n # Общая информация:\n\n\nЭтаж:\n\n                6\xa0/\xa07\n                \n            \n\n\nТип дома:\n\n            вторичка\n            \n        \n\n\nВысота потолков:\n\n                4,50\xa0м\n            \n\n\nТип продажи:\nсвободная\n\n\nОбщая площадь:\n\n151\xa0м2\n\n\n\nПлощадь комнат:\n\n20-27-18\xa0м2\n\n\n\nЖилая площадь:\n\n65\xa0м2\n\n\n\nПлощадь кухни:\n\n44\xa0м2\n\n\n\nСовмещенных санузлов:\n2\n            \n\n\nБалкон:\n–\n        \n\n\nЛифт:\n–\n\n\nПарковка:\nназемная\n\n\nВид из окна:\nулица\n\n\nРемонт:\nдизайнерский\n\n'

# Этаж и всего этажей

In [222]:
def getFloorAndNFloors(flat_page):
    try:
        table = getTableOfVariables(flat_page)
        floor = ''
        Nfloors=''
        part = re.split('Этаж:|Тип дома', table)[1]
        if '/' not in part:
            floor=part
            Nfloors = 'na'
        else:
            for i in part:
                if '/' in i:
                    break
                else:
                    floor += i
        
        if Nfloors != 'na':
            k=0
            for i in part:
                if i=='/':
                    s=k+2
                    while part[s].isdigit():
                        Nfloors+=part[s]
                        s+=1
                k+=1
            Nfloors=int("".join(Nfloors.split()))
        else:
            Nfloors = np.nan
        

        floor = "".join(floor.split())
        floor=int(floor)
    except:
        floor=np.nan
        Nfloors=np.nan

    return(floor,Nfloors)

In [223]:
getFloorAndNFloors(flat_page)

(7, 28)

In [30]:
flatStats['Floor'], flatStats['Nfloors'] = getFloorAndNFloors(flat_page)

# Тип здания//Новизна здания

In [224]:
def getBuildingType(flat_page):
    try:
        table = getTableOfVariables(flat_page)
        table = table.lower()
        part = re.split('тип дома:|тип продажи:', table)[1]
        if 'вторичка' in part:
            buildNew = 0
        else:
            buildNew = 1
        if 'кирпичный' in part or 'монолитный' in part or 'железобетонный' in part or 'жб' in part:
            buildType = 1
        else:
            buildType = 0
    except:
        buildType=np.nan
        buildNew=np.nan
    return(buildType,buildNew)

In [225]:
getBuildingType(flat_page)  #0 - second, 1 - first

(1, 0)

In [33]:
flatStats['Brick'],flatStats['New'] = getBuildingType(flat_page)
flatStats

{'Brick': 0,
 'Dist': 1.59,
 'District': 0,
 'Floor': 6,
 'New': 0,
 'Nfloors': 7,
 'Price': 88782540,
 'Rooms': 4}

# Общая площадь

In [226]:
def getTotalSquare(flat_page):
    try:
        table = getTableOfVariables(flat_page)
        part = re.split('Общая площадь:|Площадь комнат', table)[1]
        square =''
        for i in part:
            if i=='–':
                square += 'na'
                break
            if 'м' in i:
                break
            else:
                square+=i
        square = ''.join(square.split())
        if square == 'na':
            square = np.nan
        else:
            square = float(square.replace(',','.'))
    except:
        square=np.nan
    return(square)
    

In [227]:
getTotalSquare(flat_page)

263.0

In [77]:
flatStats['Totsp'] = getTotalSquare(flat_page)

# Жилая площадь

In [228]:
def getLiveSquare(flat_page):
    try:
        table = getTableOfVariables(flat_page)
        part = re.split('Жилая площадь:|Площадь кухни', table)[1]
        square =''
        for i in part:
            if i=='–':
                square += 'na'
                break
            elif 'м' in i:
                break
            else:
                square+=i
        square = ''.join(square.split())
        if square == 'na':
            square = np.nan
        else:
            square = float(square.replace(',','.'))
    except:
        square=np.nan
    return(square)

In [229]:
getLiveSquare(flat_page)

200.0

In [80]:
flatStats['Livesp'] = getLiveSquare(flat_page)

# Площадь кухни

In [244]:
def getKitchenSquare(flat_page):
    try:
        table = getTableOfVariables(flat_page)
        table = table.lower()
        #part = re.split('Жилая площадь:|Площадь кухни', table)[1]
        part = re.split('площадь кухни:|раздельных санузлов', table)[1]
        square =''
        for i in part:
            if i=='–':
                square += 'na'
                break
            elif 'м' in i:
                break
            else:
                square+=i
        square = ''.join(square.split())
        if square == 'na':
            square = np.nan
        else:
            square = float(square.replace(',','.'))
    except:
        square=np.nan
    return(square)

In [245]:
flatStats['Kitsp'] = getKitchenSquare(flat_page)
getKitchenSquare(flat_page)


nan

# Телефон

In [246]:
def getPhone(flat_page):
    try:
        table = getTableOfVariables(flat_page)
        table = table.lower()
        if 'телeфон' not in table:
            Tel = 0
            return(Tel)
        else:
            part = re.split('телефон:', table)[1]
            for i in part:
                if 'д' in i:
                    Tel = 1
                    break
                elif '-' in i or 'н' in i:
                    Tel=0
                    break       
            return(Tel)
    except:
        Tel=np.nan
        return(Tel)

In [247]:
flatStats['Tel'] = getPhone(flat_page)
flatStats

{'Bal': 1,
 'Brick': 0,
 'Dist': 20.21,
 'District': 8,
 'Floor': 10,
 'Kitsp': nan,
 'Livesp': 46.0,
 'Metrdist': 13,
 'N': 1,
 'New': 0,
 'Nfloors': 17,
 'Price': 10500000,
 'Rooms': 3,
 'Tel': 0,
 'Totsp': 76.0,
 'Walk': 1,
 'lat': 55.852935,
 'lon': 37.343892}

# Балкон

In [248]:
def getBalcony(flat_page):
    try:
        table = getTableOfVariables(flat_page)
        table = table.lower()
        if 'Бал' not in table:
            Bal = 0
        part = re.split('балкон:', table)[1]
        for i in part:
            if 'д' in i or i.isdigit():
                Bal = 1
                break
            elif '-' in i or 'н' in i:
                Bal=0
                break  
    except:
        Bal=np.nan
    return(Bal)

In [250]:
flatStats['Bal']=getBalcony(flat_page)
getBalcony(flat_page)

1

# Время от метро и способ добраться

In [251]:
def getMetro(flat_page):
    metro = flat_page.find('div', attrs={'class':'object_descr_metro'})
    metro = html_stripper(metro)
    try:
        part1 = re.split(',|мин', metro)[1]
        metro_min = ''
        for i in part1:
            if i.isdigit():
                metro_min+=i
        metro_min = "".join(metro_min.split())
        if 'пешком' in metro:
            metro_walk =1
        else:
            metro_walk =0
        metro_min=int(metro_min)
    except:
        metro_min = np.nan
        metro_walk = np.nan
    return(metro_min, metro_walk)

In [252]:
flatStats['Metrdist'], flatStats['Walk']  = getMetro(flat_page)
flatStats['N'] = 1
flatStats

{'Bal': 1,
 'Brick': 0,
 'Dist': 20.21,
 'District': 8,
 'Floor': 10,
 'Kitsp': nan,
 'Livesp': 46.0,
 'Metrdist': 13,
 'N': 1,
 'New': 0,
 'Nfloors': 17,
 'Price': 10500000,
 'Rooms': 3,
 'Tel': 0,
 'Totsp': 76.0,
 'Walk': 1,
 'lat': 55.852935,
 'lon': 37.343892}

# aaand finalyyyyy loop for all pages and all districts

In [278]:
FLATS = pd.DataFrame(columns=['N','Rooms','Price','Totsp','Livesp','Kitsp','Dist',
                             'Metrdist','Walk','Brick','Tel','Bal','Floor','Nfloors','New','District'])
total_pages = 30
total_districts = 12
districts = ['http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=152&district%5B1%5D=153&district%5B2%5D=154&district%5B3%5D=355&district%5B4%5D=356&district%5B5%5D=357&district%5B6%5D=358&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=327&district%5B10%5D=337&district%5B1%5D=328&district%5B2%5D=329&district%5B3%5D=330&district%5B4%5D=331&district%5B5%5D=332&district%5B6%5D=333&district%5B7%5D=334&district%5B8%5D=335&district%5B9%5D=336&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1',
            'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=338&district%5B1%5D=339&district%5B2%5D=340&district%5B3%5D=341&district%5B4%5D=342&district%5B5%5D=343&district%5B6%5D=344&district%5B7%5D=345&district%5B8%5D=346&district%5B9%5D=347&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1']
count = 0
for district in range(total_districts):
    links = getlinks(districts[district],total_pages)
    for link in range(len(links)):
        print(links[link])
        flat_page = getFlatPage(links[link])
        flatStats = {'N':link,'Rooms':getRoomNumber(flat_page),'Price':getPrice(flat_page),
                         'Totsp':getTotalSquare(flat_page),'Livesp':getLiveSquare(flat_page),
                         'Kitsp':getKitchenSquare(flat_page),'Dist':CenterDistance(flat_page),
                        'Metrdist':getMetro(flat_page)[0],'Walk':getMetro(flat_page)[1],
                         'Brick':getBuildingType(flat_page)[0],'Tel':getPhone(flat_page),
                         'Bal':getBalcony(flat_page),'Floor':getFloorAndNFloors(flat_page)[0],
                         'Nfloors':getFloorAndNFloors(flat_page)[1],'New':getBuildingType(flat_page)[1],
                         'lat':getCoordinates(flat_page)[0],'lon':getCoordinates(flat_page)[1],'District':district}
        FLATS = FLATS.append(flatStats, ignore_index=True)
        count +=1 

        

150644125
138640967
150090567
151140252
149031259
151004941
150346368
149031296
150640194
151004981
148794719
149565205
149849414
149300855
149788538
148112883
149727197
151005044
149031382
151005116
148800467
150910021
149031279
150459322
151005216
151005085
140980631
150260995
144380578
149876694
150599475
150678964
150679023
150977773
150720249
147655377
149031235
149431030
149213362
19220848
150684424
149213364
149031293
150974485
150545360
149808503
150645957
149390159
150118457
148829580
149311291
149396229
150977807
149031368
150684509
150684571
148956565
150233595
150858218
149031262
149063476
149031336
149031388
150945577
150236079
151120564
151005178
150833993
149385643
150977411
146275883
149231113
150043158
149031373
149031278
150638556
150405030
147872339
150833977
148677909
150206839
151085732
138051116
151005822
150425454
41938344
149462874
151005668
151110260
150975668
147398175
150418247
149031312
149933253
148800461
149031331
144408040
150118455
148257362
150389374
14

In [281]:
len(FLATS)

9360

In [279]:
FLATS.to_csv('D:/Documents/Дуня/ЦМФ/BigData/Week4/NEWS3.csv')

In [284]:
FLATS.head()

,N,Rooms,Price,Totsp,Livesp,Kitsp,Dist,Metrdist,Walk,Brick,Tel,Bal,Floor,Nfloors,New,District,lat,lon
0,0.0,4.0,30850000.0,115.0,55.5,30.0,3.41,6.0,1.0,1.0,0.0,1.0,2.0,12.0,0.0,0.0,55.766754,37.566395
1,1.0,1.0,20732000.0,56.8,NaN,NaN,3.62,5.0,1.0,1.0,0.0,1.0,8.0,13.0,1.0,0.0,55.723331,37.619971
2,2.0,NaN,532695240.0,399.0,NaN,NaN,1.40,7.0,1.0,0.0,0.0,1.0,8.0,9.0,0.0,0.0,55.749603,37.597872
3,3.0,2.0,12300.0,54.0,32.0,13.0,4.79,15.0,1.0,0.0,0.0,0.0,2.0,5.0,0.0,0.0,55.761381,37.541404
4,4.0,1.0,4585315.0,17.3,17.3,NaN,4.07,6.0,1.0,0.0,0.0,0.0,2.0,NaN,1.0,0.0,55.775753,37.671938
